<a href="https://colab.research.google.com/github/koodongh/20220217/blob/main/test_1222_%EC%BD%94%EB%93%9C%EC%88%98%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## 코드정리_1222_Oneline

In [ ]:
## 라이브러리 import

In [ ]:
import tensorflow as tf
import os
import random
import numpy as np
import cv2
from tqdm import tqdm
import shutil
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import glob
import PIL.Image
import PIL.ImageDraw
from scipy.signal import find_peaks,peak_widths

In [ ]:
## 1.첫번째임무수행  (Unet)_google colab 버전

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 3


TEST_PATH = '/content/gdrive/MyDrive/MTM_1207/test_f/'

test_ids = next(os.walk(TEST_PATH))[1]

Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

In [ ]:
# test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Resizing test images')
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
  path = TEST_PATH + id_
  img = imread(path + '/images/' + id_ + '.jpg')[:,:,:IMG_CHANNELS]
  #img = np.rot90(img,2) # 이미지회전이 필요한경우

  img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
  X_test[n] = img

print('Done!')

image_x = random.randint(0, len(test_ids))
imshow(X_test[image_x])
plt.show()
#imshow(np.squeeze(Y_train[image_x]))
#plt.show()

In [ ]:
from keras.models import load_model
model = load_model('gdrive/MyDrive/saved_tmp/test1115_04_99611.h5')

In [ ]:
preds_test = model.predict(X_test, verbose=1)
preds_test_t = (preds_test > 0.8).astype(np.uint8)

In [ ]:
##test

# 두개를 보개해주는 함수를 만들자.

nx = 1  
img = X_test[nx,:,:,:]
plt.imshow(img)

In [ ]:
plt.imshow(np.squeeze(preds_test_t[nx,:,:,:]))
mask = np.squeeze(preds_test_t[nx,:,:,:])
plt.imshow(mask)

In [ ]:
from glob import glob

folder_name = glob('/content/gdrive/MyDrive/MTM_1207/test_f/*')
file_name = glob(folder_name[1]+'/images/*')

print(folder_name)

In [ ]:
print(folder_name[0][40:])

In [ ]:
# 마스크사진만들기

def make_folder(i):

  path = '/content/gdrive/MyDrive/test_1207/'
  os.makedirs(path+folder_name[i][40:])
  os.makedirs(path+folder_name[i][40:]+'/images')
  os.makedirs(path+folder_name[i][40:]+'/masks')
  os.makedirs(path+folder_name[i][40:50]+'/masked')
  shutil.copy(folder_name[i] + '/images/'+ folder_name[i][40:]+'.jpg', path+folder_name[i][40:]+'/images' )

  save_png_Mask = path + folder_name[i][40:50] + '/masks/' + folder_name[i][40:50] + '.png'
  mask = np.squeeze(preds_test_t[i,:,:,:])
  mask = np.rot90(mask,2) # 이미지회전이 필요한경우
  plt.axis('off')
  plt.imsave(save_png_Mask,mask, cmap='gray')

  save_png_Masked = path + folder_name[i][40:50] + '/masked/' + folder_name[i][40:50] + '.png'
  file_name = glob(folder_name[i]+'/images/*')
  img2 = cv2.imread(file_name[0],cv2.IMREAD_UNCHANGED) # size1024x1024
  img_org = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
  img_org = cv2.resize(img_org,(1024,1024))
  img_org = cv2.rotate(img_org, cv2.ROTATE_90_CLOCKWISE) # 이미지회전이 필요한경우.
  print(img_org.shape)
  img3 = cv2.imread(save_png_Mask,cv2.IMREAD_UNCHANGED)
  img4 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
  th,dst = cv2.threshold(img4,0,255,cv2.THRESH_BINARY)
  dst_re = cv2.resize(dst,(1024,1024))
  print(dst_re.shape)
  masked =  cv2.bitwise_and(img_org, dst_re)
  plt.imsave(save_png_Masked,masked, cmap='gray')
  print( str(i) + " / " + str(len(folder_name)))


In [ ]:
for i in range(len(folder_name)):
  make_folder(i)

In [ ]:
## 2.두번째임무수행  이미지분석_Colorline Detect

In [ ]:
f_list = glob('/content/gdrive/MyDrive/test_1207/*')

len(f_list)

In [ ]:
# 색 모음
## while 문 외부에 두자

default = (255,255,255)

# blue_YCbCr
blue_u_1 = (85, 0, 162)
blue_u_2 = (161, 95, 184)

# lightblue_YCbCr
lightblue_u_1 = (131,63,144)
lightblue_u_2 = (226,88,152)

# blue_hsv

hsv_blue_u_1 = (86,108,215)
hsv_blue_u_2 = (97,180,255)

# lightblue_hsv

hsv_blue_l_1 = (90,36,245)
hsv_blue_l_2 = (93,82,255)

# pink_ycbcr

pink_u_1 = (117, 139, 110) 
pink_u_2 = (255, 255, 255) 

# pink_hsv
    
hsv_pink_u_1 = (106, 0, 232)
hsv_pink_u_2 = (172, 29, 255)

# yellow_ycbcr

yellow_u_1 = (126, 134, 0)
yellow_u_2 = (255, 255, 108)

# lightyellow

lightyellow_u_1 = (178, 122, 51)   
lightyellow_u_2 = (255, 206, 123) 

# 4. Orange

# Ornage_ycbcr

orange_u_1 = (85, 134, 77)
orange_u_2 = (164, 198, 120)

# Orange_hsv

hsv_orange_u_1 = (0,53,80)
hsv_orange_u_2 = (12,95,130)

# 5. Green

# green_ycbcr

green_u_1 = (89, 0, 0)
green_u_2 = (173, 112, 131)

# lightgreen_ycbcr

lightgreen_u_1 = (195, 0, 51)  
lightgreen_u_2 = (255, 105, 138)

# 7. Purple

# purple_ycbcr
purple_u_1 = (64, 106, 155)   # 150~155 사이로 조절 조금만 낮춰야할듯
purple_u_2 = (104, 145, 255)

# purple_hsv

hsv_purple_u_1 = (110,86,97)
hsv_purple_u_2 = (149,155,255)

# lightpurple_right_hsv

hsv_purple_light_right_u_1 = (105,50,207) #60->#30 105를 106으로 수정하지 말것
hsv_purple_light_right_u_2 = (115,106,255) #
hsv_purple_light_right_l_1 = (106,10,220) # 105 # 108->106
hsv_purple_light_right_l_2 = (122,80,255)

white_u_1 = (77,0,247)
white_u_2 = (100,30,255)


In [ ]:
def make_mask_hsv(upper_1, upper_2, lower_1, lower_2):

    upper_temp = cv2.inRange(img_hsv, upper_1, upper_2)
    lower_temp = cv2.inRange(img_hsv, lower_1, lower_2)
    
    mask_pre = cv2.addWeighted(upper_temp, 1.0, lower_temp, 1.0, 0.0)
    mask_li = [1 if mask_pre[:,i-2:i].sum() > 30000 else 0 for i in range(len(mask_pre))]
    mask_temp = np.array(mask_pre * mask_li,dtype='uint8')
    
    return mask_temp

In [ ]:
def make_mask_ycbcr(upper_1, upper_2, lower_1, lower_2):

    upper_temp = cv2.inRange(img_yCbCr, upper_1, upper_2)
    lower_temp = cv2.inRange(img_yCbCr, lower_1, lower_2)
    
    mask_pre = cv2.addWeighted(upper_temp, 1.0, lower_temp, 1.0, 0.0)
    mask_li = [1 if mask_pre[:,i-2:i].sum() > 30000 else 0 for i in range(len(mask_pre))]
    mask_temp = np.array(mask_pre * mask_li,dtype='uint8')
    
    return mask_temp

In [ ]:
def show_three(dst1,dst2,dst3,tread_width, num):
    dst1 = cv2.cvtColor(dst1, cv2.COLOR_BGR2RGB)
    dst2 = cv2.cvtColor(dst2, cv2.COLOR_BGR2RGB)
    dst3 = cv2.cvtColor(dst3, cv2.COLOR_BGR2RGB)

    font1 = {'family': 'serif',
      'color':  'red',
      'weight': 'normal',
      'size': 17}
        
    plt.figure(figsize=(15,15))
    ax1 = plt.subplot(1, 3, 1)
    ax1.set_title('Original') 
    plt.text(100,100,"Tread size : {} mm".format(tread_width), fontdict=font1)
    plt.text(100,170, 'Colorline : ', fontdict=font1)
    plt.imshow(dst1)
    
    
    ax2 = plt.subplot(1, 3, 2)
    ax2.set_title('Tread_Extract')
    plt.imshow(dst2)

    ax3 = plt.subplot(1, 3, 3)
    ax3.set_title('Color_Detect')
    plt.imshow(dst3)
    
    #plt.suptitle(num)
    print('num : ',num)
    plt.show()

In [ ]:
def colorcheck(x):

  if x == 'Blue':
    mask_blue = make_mask_ycbcr(blue_u_1,blue_u_2, default, default)
    mask_lightblue = make_mask_ycbcr(lightblue_u_1,lightblue_u_2, default, default)
    mask_blue_hsv = make_mask_hsv(hsv_blue_u_1,hsv_blue_u_2, hsv_blue_l_1, hsv_blue_l_2)
    img_mask_blue_01 = cv2.addWeighted(mask_blue, 1.0, mask_lightblue, 1.0, 0.0)
    img_mask_tmp = cv2.addWeighted(img_mask_blue_01, 1.0, mask_blue_hsv, 1.0, 0.0)
    
  if x == 'Pink':
    mask_pink = make_mask_ycbcr(pink_u_1,pink_u_2, default, default)
    mask_pink_hsv = make_mask_hsv(hsv_pink_u_1,hsv_pink_u_2, default, default)
    img_mask_tmp = cv2.addWeighted(mask_pink, 1.0, mask_pink_hsv, 1.0, 0.0)

  if x == 'Yellow':  
    mask_yellow = make_mask_ycbcr(yellow_u_1,yellow_u_2, default, default)   
    mask_lightyellow = make_mask_ycbcr(lightyellow_u_1,lightyellow_u_2, default, default)
    img_mask_tmp = cv2.addWeighted(mask_yellow, 1.0, mask_lightyellow, 1.0, 0.0)

  if x == 'Orange':
    mask_orange = make_mask_ycbcr(orange_u_1,orange_u_2, default, default)
    mask_orange_hsv = make_mask_hsv(hsv_orange_u_1,hsv_orange_u_2, default, default)
    img_mask_tmp =  cv2.addWeighted(mask_orange, 1.0, mask_orange_hsv, 1.0, 0.0)
     
  if x == 'Green':
    mask_green = make_mask_ycbcr(green_u_1,green_u_2, default, default)
    mask_lightgreen = make_mask_ycbcr(lightgreen_u_1,lightgreen_u_2, default, default)     
    img_mask_tmp = cv2.addWeighted(mask_green, 1.0, mask_lightgreen, 1.0, 0.0)

  if x == 'Purple':
    mask_purple = make_mask_ycbcr(purple_u_1,purple_u_2, default, default)
    mask_purple_hsv = make_mask_hsv(hsv_purple_u_1,hsv_purple_u_2, default, default)
    mask_lightpurple_right_hsv = make_mask_hsv(hsv_purple_light_right_u_1,hsv_purple_light_right_u_2, hsv_purple_light_right_l_1, hsv_purple_light_right_l_2)
    img_mask_purple_01 = cv2.addWeighted(mask_purple, 1.0, mask_purple_hsv, 1.0, 0.0)
    img_mask_tmp = cv2.addWeighted(img_mask_purple_01, 1.0, mask_lightpurple_right_hsv, 1.0, 0.0)

  if x == 'White':
    img_mask_white = make_mask_hsv(white_u_1,white_u_2, default, default)

    cnt = 30

    cnt_area = [i*cnt for i in range(200) if (img_mask_white[:,i*cnt:(i+1)*cnt].sum()) > 0]
    cnt_del = [cnt_area[i] for i in range(len(cnt_area)) if (img_mask_white[:,cnt_area[i]:cnt_area[i]+cnt].sum(axis=1) > 1).sum() < 700]
    cnt_nor = [cnt_area[i] for i in range(len(cnt_area)) if (img_mask_white[:,cnt_area[i]:cnt_area[i]+cnt].sum(axis=1) > 1).sum() > 700]    

    for i in range(len(cnt_del)):
        if cnt_del[i] + 30 not in cnt_nor: 
            img_mask_white[:,cnt_del[i]:cnt_del[i]+30] = 0
    img_mask_tmp = img_mask_white

  return img_mask_tmp



In [ ]:
## 1. 색상 카운팅하기

num =  1
mylist = {}

mask_raw = cv2.imread(f_list[num] + '/masks/' + f_list[num][34:] + '.png')
img_raw = cv2.imread(f_list[num] + '/images/' + f_list[num][34:] + '.jpg')
masked_raw = cv2.imread(f_list[num] + '/masked/' + f_list[num][34:] + '.png')
masked = masked_raw

## image name
print('Filename : ', f_list[num][34:])

img_color = masked
img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV) 
img_yCbCr = cv2.cvtColor(img_color, cv2.COLOR_BGR2YCR_CB)



# 1. Blue
mask_blue = make_mask_ycbcr(blue_u_1,blue_u_2, default, default)
blue_ycbcr_sum = mask_blue.sum()
mask_lightblue = make_mask_ycbcr(lightblue_u_1,lightblue_u_2, default, default)
lightblue_sum = mask_lightblue.sum()
img_mask_1 = cv2.addWeighted(mask_blue, 1.0, mask_lightblue, 1.0, 0.0)
mask_blue_hsv = make_mask_hsv(hsv_blue_u_1,hsv_blue_u_2, hsv_blue_l_1, hsv_blue_l_2)
blue_hsv_sum = mask_blue_hsv.sum()
blue_sum = blue_ycbcr_sum + lightblue_sum + blue_hsv_sum
mylist['Blue']=blue_sum
img_mask_2 = cv2.addWeighted(img_mask_1, 1.0, mask_blue_hsv, 1.0, 0.0)
img_mask_blue_01 = cv2.addWeighted(mask_blue, 1.0, mask_lightblue, 1.0, 0.0)
img_mask_blue = cv2.addWeighted(img_mask_blue_01, 1.0, mask_blue_hsv, 1.0, 0.0)
   
# 2. Pink
mask_pink = make_mask_ycbcr(pink_u_1,pink_u_2, default, default)
pink_ycbcr_sum = mask_pink.sum()
img_mask_3 = cv2.addWeighted(img_mask_2, 1.0, mask_pink, 1.0, 0.0)
mask_pink_hsv = make_mask_hsv(hsv_pink_u_1,hsv_pink_u_2, default, default)
mask_pink_hsv_sum = mask_pink_hsv.sum()
pink_sum = pink_ycbcr_sum + mask_pink_hsv_sum
mylist['Pink'] = pink_sum
img_mask_4 = cv2.addWeighted(img_mask_3, 1.0, mask_pink_hsv, 1.0, 0.0)
img_mask_pink = cv2.addWeighted(mask_pink, 1.0, mask_pink_hsv, 1.0, 0.0)
 
# 3. Yellow
mask_yellow = make_mask_ycbcr(yellow_u_1,yellow_u_2, default, default)
yellow_ycbcr_sum = mask_yellow.sum()
img_mask_5 = cv2.addWeighted(img_mask_4, 1.0, mask_yellow, 1.0, 0.0)
mask_lightyellow = make_mask_ycbcr(lightyellow_u_1,lightyellow_u_2, default, default)
lightyellow_sum = mask_lightyellow.sum()
yellow_sum = yellow_ycbcr_sum + lightyellow_sum
mylist['Yellow'] = yellow_sum
img_mask_6 = cv2.addWeighted(img_mask_5, 1.0, mask_lightyellow, 1.0, 0.0)
img_mask_yellow = cv2.addWeighted(mask_yellow, 1.0, mask_lightyellow, 1.0, 0.0)
  
# 4. Orange
mask_orange = make_mask_ycbcr(orange_u_1,orange_u_2, default, default)
orange_ycbcr_sum = mask_orange.sum()
img_mask_7 = cv2.addWeighted(img_mask_6, 1.0, mask_orange, 1.0, 0.0)
mask_orange_hsv = make_mask_hsv(hsv_orange_u_1,hsv_orange_u_2, default, default)
orange_hsv_sum = mask_orange_hsv.sum()
orange_sum_01 = orange_ycbcr_sum + orange_hsv_sum
orange_sum = orange_ycbcr_sum + orange_hsv_sum + lightyellow_sum

if yellow_sum > orange_sum:
  mylist['Orange'] = orange_sum_01
elif yellow_sum == orange_sum:
  mylist['Orange'] = orange_sum_01      
elif orange_sum_01 > 1000000:  
  mylist['Yellow'] = 0
  mylist['Pink'] = pink_sum - 1000000 
  mylist['Orange'] = orange_sum_01
if yellow_sum > 2000000 and orange_sum>500000:
  mylist['Orange'] = 0
elif orange_sum_01 < 200000:
  mylist['Orange'] = 0
else:
  mylist['Orange'] = orange_sum  

img_mask_8 = cv2.addWeighted(img_mask_7, 1.0, mask_orange_hsv, 1.0, 0.0)
img_mask_orange =  cv2.addWeighted(mask_orange, 1.0, mask_orange_hsv, 1.0, 0.0)

# 5. Green     
mask_green = make_mask_ycbcr(green_u_1,green_u_2, default, default)
green_ycbcr_sum = mask_green.sum()
img_mask_9 = cv2.addWeighted(img_mask_8, 1.0, mask_green, 1.0, 0.0)
mask_lightgreen = make_mask_ycbcr(lightgreen_u_1,lightgreen_u_2, default, default)
lightgreen_sum = mask_lightgreen.sum()
green_sum = green_ycbcr_sum + lightgreen_sum
mylist['Green']= green_sum
img_mask_10 = cv2.addWeighted(img_mask_9, 1.0, mask_lightgreen, 1.0, 0.0)       
img_mask_green = cv2.addWeighted(mask_green, 1.0, mask_lightgreen, 1.0, 0.0)

# 6. Purple
mask_purple = make_mask_ycbcr(purple_u_1,purple_u_2, default, default)
purple_sum_ycbcr = mask_purple.sum()
img_mask_11 = cv2.addWeighted(img_mask_10, 1.0, mask_purple, 1.0, 0.0)
mask_purple_hsv = make_mask_hsv(hsv_purple_u_1,hsv_purple_u_2, default, default)
purple_hsv_sum = mask_purple_hsv.sum()  
img_mask_12 = cv2.addWeighted(img_mask_11, 1.0, mask_purple_hsv, 1.0, 0.0)
mask_lightpurple_right_hsv = make_mask_hsv(hsv_purple_light_right_u_1,hsv_purple_light_right_u_2, hsv_purple_light_right_l_1, hsv_purple_light_right_l_2)
mask_lightpurple_right_sum = mask_lightpurple_right_hsv.sum()
purple_sum = purple_sum_ycbcr + purple_hsv_sum + mask_lightpurple_right_sum
if pink_sum > purple_sum and purple_sum<900000: 
  purple_sum = 0 
else:
  purple_sum = purple_sum 
mylist['Purple']= purple_sum   
img_mask_13 = cv2.addWeighted(img_mask_12, 1.0, mask_lightpurple_right_hsv, 1.0, 0.0)
img_mask_purple_01 = cv2.addWeighted(mask_purple, 1.0, mask_purple_hsv, 1.0, 0.0)
img_mask_purple = cv2.addWeighted(img_mask_purple_01, 1.0, mask_lightpurple_right_hsv, 1.0, 0.0)

# 7. White
img_mask_white = make_mask_hsv(white_u_1,white_u_2, default, default)
cnt = 30
cnt_area = [i*cnt for i in range(200) if (img_mask_white[:,i*cnt:(i+1)*cnt].sum()) > 0]
cnt_del = [cnt_area[i] for i in range(len(cnt_area)) if (img_mask_white[:,cnt_area[i]:cnt_area[i]+cnt].sum(axis=1) > 1).sum() < 700]
cnt_nor = [cnt_area[i] for i in range(len(cnt_area)) if (img_mask_white[:,cnt_area[i]:cnt_area[i]+cnt].sum(axis=1) > 1).sum() > 700]    
for i in range(len(cnt_del)):
  if cnt_del[i] + 30 not in cnt_nor: 
    img_mask_white[:,cnt_del[i]:cnt_del[i]+30] = 0
white_sum = img_mask_white.sum()
mylist['White']= white_sum   
img_mask_13 = cv2.addWeighted(img_mask_13, 1.0, img_mask_white, 1.0, 0.0)
#img_resultㄱ
img_result = cv2.bitwise_and(img_color, img_color, mask = img_mask_13)
d = {'color': ['Blue', 'Pink','Yellow','Orange','Green','White','Purple'], 'sum': [blue_sum, pink_sum,yellow_sum,orange_sum,green_sum,white_sum,purple_sum]}
df = pd.DataFrame(data = d)
df = df.sort_values(by='sum', ascending=False)
print(df)print('#######Result#######')
colorline = sorted(mylist.items(), key=lambda x: x[1],reverse=True)
color_list = []
color_sum_list = []
color_out = []
print(colorline)
# 1. color추출값판단
for i in colorline:
  if i[1] > 150000 :  # min range설정 #다시 확인하기 
    color_list.append(i[0])
    color_sum_list.append(i[1])
  else:
    pass
no = ['No color']
print('color_sum_list:',color_sum_list)

In [ ]:
ㄱ## 2. 추출된 color 판별



if len(color_list) >= 3 and color_sum_list[2]>500000 :  #1000000->500000
  #print(color_sum_list[2])
  color_out.append(color_list[0])
  color_out.append(color_list[1])
  color_out.append(color_list[2])
  #print(color_list[2])
elif len(color_list) >= 2:
  color_out.append(color_list[0])
  color_out.append(color_list[1])
elif len(color_list) == 1:
  color_out.append(color_list[0])
elif len(color_list) == 0:
  color_out.append(no[0]) # 이게 적용이안됬다.

## 3. color_out 출력

print("colorout  :",color_out)
# mask_raw
img_01 = mask_raw
    # 구조화 요소 커널, 사각
    # 3,1 5,1
    #형 (3x3) 생성 ---①
k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,2))
    # 팽창 연산 적용 ---②
dst_dil_01 = cv2.dilate(img_01, k) 
img_peak_01 = cv2.cvtColor(img_01, cv2.COLOR_BGR2GRAY)
line_1 = img_peak_01[400,:] 
peaks, properties = find_peaks(line_1, distance =7) # min 5
print(f'index of each peaks(treadsize) : {peaks} ')
results_width = peak_widths(line_1, peaks, rel_height=0.5)
tread_width = results_width[0]
tread_width =  tread_width[0]
print(tread_width)
print('count peak(treadsize) : ',len(peaks))
#masked_raw

img_02 = img_result

k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
dst_dil_02 = cv2.dilate(img_02, k)
line_count = 800
img_peak_02 = cv2.cvtColor(dst_dil_02, cv2.COLOR_BGR2GRAY)
line_2 = img_peak_02[line_count,:]
peaks_result, properties_result = find_peaks(line_2, distance =7) # min 5
print(f'index of each peaks(linedetect) : {peaks_result} ')
print('count peak(linedetect) : ',len(peaks_result))

# 기준선 수정 logic

line_count_re = 800
if len(color_out) == 2 and (len(peaks_result) ==0 or len(peaks_result) == 1) :
  try:
    while True:
      line_count_re+=1
      img_peak_02 = cv2.cvtColor(dst_dil_02, cv2.COLOR_BGR2GRAY)
        
      line_2 = img_peak_02[line_count_re,:]          
      peaks_result, properties_result = find_peaks(line_2, distance =7) 
      print(f'index of each peaks : {peaks_result} ')
      print('count peak : ',len(peaks_result))
      if len(peaks_result) == 1:
        line_count_re += 1
      if len(peaks_result) == 2:
        break
      if line_count_re > 1000 :
        break
        
  except IndexError as e:
    print('line_count를 수정해주세요',e)      
  else:
    pass
  finally:
    print('line_count수정완료.',line_count_re)

if line_count != line_count_re:

  line_count_tread = int(line_count_re/2)
  img_01 = mask_raw
  k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,2))
  dst_dil_01 = cv2.dilate(img_01, k)  
  img_peak_01 = cv2.cvtColor(img_01, cv2.COLOR_BGR2GRAY)
  line_1 = img_peak_01[line_count_tread,:]
  peaks, properties = find_peaks(line_1, distance =7) # min 5
  print(f'index of each peaks(treadsize)(re) : {peaks} ')
  results_width = peak_widths(line_1, peaks, rel_height=0.5)
  tread_width = results_width[0]
  tread_width =  tread_width[0]
  print(tread_width)
  print('count peak(treadsize)(re) : ',len(peaks))

  #masked_raw
  img_02 = img_result
  k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
  dst_dil_02 = cv2.dilate(img_02, k)
  img_peak_02 = cv2.cvtColor(dst_dil_02, cv2.COLOR_BGR2GRAY)
  line_2 = img_peak_02[line_count_re,:]      
  peaks_result, properties_result = find_peaks(line_2, distance =7) # min 5
  print(f'index of each peaks(linedetect) : {peaks_result} ')
  print('count peak(linedetect) : ',len(peaks_result))

if len(color_out) >= 2 and color_sum_list[1]<500000:
  print("case:len(color_out) >= 2 and color_sum_list[1]<500000")
  #lineExtract②
  x1 = color_out[0]
  img_mask_tmp = colorcheck(x1)
  img_001 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
  k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
  dst_dil_001 = cv2.dilate(img_001, k)  
  img_peak_001 = cv2.cvtColor(dst_dil_001, cv2.COLOR_BGR2GRAY)
  line_001 = img_peak_001[line_count,:]
  peaks_01, properties_01 = find_peaks(line_001, distance =7) # min 5
  print(f'index of each peaks : {peaks_01} ')
  print('count peak : ',len(peaks_01))
  output_01 = (peaks_01[0] - ((peaks[0] - tread_width/2)*2))/2

  x2 = color_out[1]
  img_mask_tmp = colorcheck(x2)
  img_002 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
  k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
  dst_dil_002 = cv2.dilate(img_002, k)  
  img_peak_002 = cv2.cvtColor(dst_dil_002, cv2.COLOR_BGR2GRAY)
  line_002 = img_peak_002[line_count,:]  
  peaks_02, properties_02 = find_peaks(line_002, distance =7) # min 5
  print(f'index of each peaks : {peaks_02} ')
  print('count peak : ',len(peaks_02))
  if len(peaks_02) >= 1:
    output_02 = (peaks_02[0] - ((peaks[0] - tread_width/2)*2))/2
  else:
    output_02 = '+NotFind'


if len(color_out) == 1 :
  if color_out[0] == 'No color' :
    output_01 = ''
  if color_out[0] != 'No color' :
    if len(peaks_result) == 1 :
      #lineExtract①
      x = color_out[0]
      img_mask_tmp = colorcheck(x)
      img_001 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
      k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
      dst_dil_001 = cv2.dilate(img_001, k)
  
      img_peak_001 = cv2.cvtColor(dst_dil_001, cv2.COLOR_BGR2GRAY)
      line_001 = img_peak_001[line_count,:]  
      peaks_01, properties_01 = find_peaks(line_001, distance =7) # min 5
      print(f'index of each peaks : {peaks_01} ')
      print('count peak : ',len(peaks_01))
      print(peaks[0])
      print(((peaks[0] - tread_width/2)*2))
      output_01 = (peaks_01[0] - ((peaks[0] - tread_width/2)*2))/2

    elif len(peaks_result) == 2:
      x = color_out[0]
      img_mask_tmp = colorcheck(x)
      img_001 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
      k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
      dst_dil_001 = cv2.dilate(img_001, k)
      
      img_peak_001 = cv2.cvtColor(dst_dil_001, cv2.COLOR_BGR2GRAY)
      line_001 = img_peak_001[line_count,:]  
      peaks_01, properties_01 = find_peaks(line_001, distance =7) # min 5
      print(f'index of each peaks : {peaks_01} ')
      print('count peak : ',len(peaks_01))
      print(peaks[0])
      print(((peaks[0] - tread_width/2)*2))
      output_01 = (peaks_01[0] - ((peaks[0] - tread_width/2)*2))/2
      output_02 = (peaks_01[1] - ((peaks[0] - tread_width/2)*2))/2

elif len(peaks_result) == 2 and len(color_out) >= 2 :

  print('case:len(peaks_result==2) and len(color_out) >= 2')
  x = color_out[0]
  img_mask_tmp = colorcheck(x)
  img_001 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
  k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
  dst_dil_001 = cv2.dilate(img_001, k)
  
  img_peak_001 = cv2.cvtColor(dst_dil_001, cv2.COLOR_BGR2GRAY)
  line_001 = img_peak_001[line_count,:]  
  peaks_01, properties_01 = find_peaks(line_001, distance =7) # min 5
  print(f'index of each peaks : {peaks_01} ')
  print('count peak : ',len(peaks_01))

  if len(peaks_01) == 1:
    output_01 = (peaks_01[0] - ((peaks[0] - tread_width/2)*2))/2
  elif len(peaks_01) == 2:
    output_01 = (peaks_01[0] - ((peaks[0] - tread_width/2)*2))/2
    output_02 = (peaks_01[1] - ((peaks[0] - tread_width/2)*2))/2

  x2 = color_out[1]
  img_mask_tmp = colorcheck(x2)
  img_002 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
  k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
  dst_dil_002 = cv2.dilate(img_002, k)  
  img_peak_002 = cv2.cvtColor(dst_dil_002, cv2.COLOR_BGR2GRAY)
  line_002 = img_peak_002[line_count,:]  
  peaks_02, properties_02 = find_peaks(line_002, distance =7) # min 5
  print(f'index of each peaks : {peaks_02} ')
  print('count peak : ',len(peaks_02))

  if len(peaks_02) == 1 and len(peaks_01) == 1:
    output_02 = (peaks_02[0] - ((peaks[0] - tread_width/2)*2))/2
  elif len(peaks_02) == 2 and len(peaks_01) == 1:
    x1 = abs(peaks_02[0] - ((peaks[0] - tread_width/2)*2))/2
    x2 = abs(peaks_02[1] - ((peaks[0] - tread_width/2)*2))/2
    if abs(x2-x1) >= 0:
      x2 = output_02
    elif abs(x1-x2) > 0:
      x1 = output_01

elif len(peaks_result) == 2 and (len(color_out) == 1 or color_sum_list[0]>1800000):
  print('len(peaks_result) == 2 and (len(color_out) == 1 or color_sum_list[0]>1800000)')

  x = color_out[0]
  img_mask_tmp = colorcheck(x)
  img_001 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
  k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
  dst_dil_001 = cv2.dilate(img_001, k)
  
  img_peak_001 = cv2.cvtColor(dst_dil_001, cv2.COLOR_BGR2GRAY)
  line_001 = img_peak_001[line_count,:]  
  peaks_01, properties_01 = find_peaks(line_001, distance =7) # min 5
  print(f'index of each peaks : {peaks_01} ')
  print('count peak : ',len(peaks_01))
  print(peaks[0])
  print(((peaks[0] - tread_width/2)*2))
  output_01 = (peaks_01[0] - ((peaks[0] - tread_width/2)*2))/2
  output_02 = (peaks_01[1] - ((peaks[0] - tread_width/2)*2))/2
elif len(peaks_result) == 1 and len(color_out) == 3 and color_out[0] == 'Orange' :
  x = color_out[0]
  img_mask_tmp = colorcheck(x)
  img_001 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
  k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
  dst_dil_001 = cv2.dilate(img_001, k) 
  img_peak_001 = cv2.cvtColor(dst_dil_001, cv2.COLOR_BGR2GRAY)
  line_001 = img_peak_001[line_count,:]  
  peaks_01, properties_01 = find_peaks(line_001, distance =7) # min 5
  print(f'index of each peaks : {peaks_01} ')
  print('count peak : ',len(peaks_01))
  output_01 = (peaks_01[0] - ((peaks[0] - tread_width/2)*2))/2



elif len(peaks_result) == 3:

  if len(color_out) == 3 and color_sum_list[2]>500000 and (color_out[0] != 'Pink'):

    print('case:len(color_out) == 3 and color_sum_list[2]>1000000') 
    x1 = color_out[0]
    img_mask_tmp = colorcheck(x1)
    img_001 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
    dst_dil_001 = cv2.dilate(img_001, k)  
    img_peak_001 = cv2.cvtColor(dst_dil_001, cv2.COLOR_BGR2GRAY)
    line_001 = img_peak_001[line_count,:]
    peaks_01, properties_01 = find_peaks(line_001, distance =7) # min 5
    print(f'index of each peaks : {peaks_01} ')
    print('1count peak : ',len(peaks_01))
    output_01 = (peaks_01[0] - ((peaks[0] - tread_width/2)*2))/2

    x2 = color_out[1]
    img_mask_tmp = colorcheck(x2)
    img_002 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
    dst_dil_002 = cv2.dilate(img_002, k)  
    img_peak_002 = cv2.cvtColor(dst_dil_002, cv2.COLOR_BGR2GRAY)
    line_002 = img_peak_002[line_count,:]  
    peaks_02, properties_02 = find_peaks(line_002, distance =7) # min 5
    print(f'index of each peaks : {peaks_02} ')
    print('2count peak : ',len(peaks_02))
    output_02 = (peaks_02[0] - ((peaks[0] - tread_width/2)*2))/2
    x3 = color_out[2]
    img_mask_tmp = colorcheck(x3)
    img_003 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
    dst_dil_003 = cv2.dilate(img_003, k)  
    img_peak_003 = cv2.cvtColor(dst_dil_003, cv2.COLOR_BGR2GRAY)
    line_003 = img_peak_003[line_count,:]  
    peaks_03, properties_03 = find_peaks(line_003, distance =7) # min 5
    print(f'index of each peaks : {peaks_03} ')
    print('3count peak : ',len(peaks_03))
    output_03 = (peaks_03[0] - ((peaks[0] - tread_width/2)*2))/2

  elif len(color_out) == 2 and color_sum_list[0]>2000000: #오렌지 2줄 /핑크선하나
    print('case:len(color_out) == 2 and color_sum_list[0]>2000000')
      #lineExtract③
    print('첫번째 :',color_out[0])
    x1 = color_out[0]
    img_mask_tmp = colorcheck(x1)
    img_001 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
    dst_dil_001 = cv2.dilate(img_001, k)  
    img_peak_001 = cv2.cvtColor(dst_dil_001, cv2.COLOR_BGR2GRAY)
    line_001 = img_peak_001[line_count,:]
    peaks_01, properties_01 = find_peaks(line_001, distance =7) # min 5
    print('peak갯수 : ',len(peaks_01))
    if len(peaks_01) == 3: # pink peak갯수가 3개가 뜬다. 그래서 순서대로 하나씩패치
      print('orangecase:len(peaks_01) == 3')
      print(f'1index of each peaks : {peaks_01[0]} ')
      print(f'2index of each peaks : {peaks_01[1]} ')
      print(f'3index of each peaks : {peaks_01[2]} ')

      output_01 = (peaks_01[0] - ((peaks[0] - tread_width/2)*2))/2            
      output_02 = (peaks_01[1] - ((peaks[0] - tread_width/2)*2))/2
      output_03 = (peaks_01[2] - ((peaks[0] - tread_width/2)*2))/2

    elif len(peaks_01) == 2:

      print('orangecase:len(peaks_01) == 2')
      print(f'index of each peaks : {peaks_01[0]} ')
      print(f'index of each peaks : {peaks_01[1]} ')
      output_01 = (peaks_01[0] - ((peaks[0] - tread_width/2)*2))/2     # 1st Orange       
      output_03 = (peaks_01[1] - ((peaks[0] - tread_width/2)*2))/2     # 2nd Orange
      x2 = color_out[1]
      img_mask_tmp = colorcheck(x2)
      img_002 = cv2.bitwise_and(img_color, img_color, mask = img_mask_tmp)
      k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,1))
      dst_dil_002 = cv2.dilate(img_002, k)  
      img_peak_002 = cv2.cvtColor(dst_dil_002, cv2.COLOR_BGR2GRAY)
      print('두번째 :  ',color_out[1]) 
      line_002 = img_peak_002[line_count,:]  
      peaks_02, properties_02 = find_peaks(line_002, distance =7) # min 5
      print('두번째 cnt : ',len(peaks_02) )
      print('두번째peak 위치',peaks_02[1])
      if peaks_02[1] != peaks_result[2]:
        output_02 = (peaks_02[1] - ((peaks[0] - tread_width/2)*2))/2
      else:
        output_02 = (peaks_result[1] - ((peaks[0] - tread_width/2)*2))/2



In [ ]:
## 3. 결과출력
print(len(color_out))
dst1 = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
dst2 = cv2.cvtColor(masked, cv2.COLOR_BGR2RGB)
dst3 = cv2.cvtColor(dst_dil_02, cv2.COLOR_BGR2RGB)
font1 = {'family': 'serif',
         'color':  'red',
         'weight': 'bold',
         'size': 13}
font2 = {'family': 'serif',
         'color':  'red',
         'weight': 'normal',
         'size': 17}
props = dict(boxstyle='round', facecolor='white', alpha=1)

plt.figure(figsize=(15,15))
ax1 = plt.subplot(1, 3, 1)
plt.text(50,350,"                                          \n                                          \n                                          \n", fontdict=font2, bbox = props)
ax1.set_title('Original') 
plt.text(70,100,"Tread size : {} mm ".format(tread_width), fontdict=font1)
plt.text(70,170, 'Color line : ', fontdict=font1)

if len(color_out) == 1 :
  if len(peaks_result) == 0:
    plt.text(430,170, "{}".format(no[0]), fontdict=font1)
  if len(peaks_result) == 1:
    plt.text(430,170, "{} +{}".format(color_out[0],output_01), fontdict=font1)
  if len(peaks_result) == 2:
    plt.text(430,170, "1st{} +{}".format(color_out[0],output_01), fontdict=font1)
    plt.text(430,250, "2nd{} +{}".format(color_out[0],output_02), fontdict=font1)
elif len(peaks_result) == 3 and len(color_out) == 2 and color_sum_list[0]>1000000 :
  plt.text(430,170, "1stOrnage +{}".format(output_01), fontdict=font1)
  plt.text(430,250, "Pink +{}".format(output_02), fontdict=font1)
  plt.text(430,330, "2ndOrange +{}".format(output_03), fontdict=font1)
elif len(peaks_result) == 3 and len(color_out) >= 2 and color_out[2] == 'Yellow' and ( color_out[0] == 'Pink' or  color_out[0] == 'Orange') : # 오렌지 두개/핑크하나
  plt.text(430,170, "1stOrnage +{}".format(output_01), fontdict=font1)
  plt.text(430,250, "Pink +{}".format(output_02), fontdict=font1)
  plt.text(430,330, "2ndOrange +{}".format(output_03), fontdict=font1)
elif len(color_out) == 2 :

  if color_out[0] =='Orange' and len(peaks_result) == 1:
    plt.text(430,170, "{} +{}".format(color_out[0],output_01), fontdict=font1)
  else:
    plt.text(430,170, "{} +{}".format(color_out[0],output_01), fontdict=font1)
    plt.text(430,250, "{} +{}".format(color_out[1],output_02), fontdict=font1)
elif len(peaks_result) == 1 and len(color_out) == 3 and color_out[0] == 'Orange' :
    plt.text(430,170, "{} +{}".format(color_out[0],output_01), fontdict=font1)
elif len(color_out) >= 3 and color_sum_list[2]>1000000:
  plt.text(430,170, "{} +{}".format(color_out[0],output_01), fontdict=font1)
  plt.text(430,250, "{} +{}".format(color_out[1],output_02), fontdict=font1)
  plt.text(430,330, "{} +{}".format(color_out[2],output_03), fontdict=font1)
elif len(peaks_result) == 3 and len(color_out) >= 3 and color_sum_list[2]>500000: #1000000
  plt.text(430,170, "{} +{}".format(color_out[0],output_01), fontdict=font1)
  plt.text(430,250, "{} +{}".format(color_out[1],output_02), fontdict=font1)
  plt.text(430,330, "{} +{}".format(color_out[2],output_03), fontdict=font1)
     
plt.imshow(dst1)
  
ax2 = plt.subplot(1, 3, 2)
ax2.set_title('Tread_Extract')
plt.imshow(dst2)

ax3 = plt.subplot(1, 3, 3)
ax3.set_title('Color_Detect')
plt.imshow(dst3)

print('num : ',num)
    
  
# 모델저장하기
    
#merged = np.hstack((dst1, dst2,dst3))
#plt.suptitle(num)
#save_tmp = '/content/gdrive/MyDrive/tmp_1207/'+f_list[num][36:]+'.jpg'
#plt.savefig(save_tmp,dpi = 100)

#출력

plt.show()
